In [0]:
!pip install transformers

In [0]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [0]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [0]:
#models: 
#monologg/biobert_v1.1_pubmed, cased
#bert-base-cased
#bert-base-uncased
#adamlin/ClinicalBert_all_notes, uncased
BERT_MODEL = 'monologg/biobert_v1.1_pubmed'
UNCASED_FLAG = False

#data path, data is loaded from the mounted Google Drive
data_path = "/content/drive/My Drive/concept_assertion_relation_training_data"

#max length of input sequence:
MAX_LEN = 128

In [0]:
#add tokenizer
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL, do_lower_case = UNCASED_FLAG)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


In [0]:
#load model first:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    BERT_MODEL, # Use the 12-layer BERT model, with an cased/uncased vocab.
    num_labels = 8, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
#build conept keys
#connection files can be taken as gold standard for the entities.
#get .con files from both BETH and PARTNERS data:
import os
import re
from glob import glob
concept_files = [y for x in os.walk(data_path) for y in glob(os.path.join(x[0], '*.con'))]
#read all files into a dictionary, to be used while training.
concept_dict = {}
for file in concept_files:
  with open(file, 'r') as f:
    lines = f.readlines()
    for line in lines:
      line = line.replace('\n','')
      if UNCASED_FLAG:
        line = str.lower(line)

      #regex for parsing out the concept and gold standard tag:
      matches = re.findall(r'"([^"]*)"', line)
      #print(matches)
      if len(matches) == 2:
        concept = matches[0]
        tag = matches[1]
        #should raise error if key already exist?
        concept_dict[concept] = tag

print(len(concept_dict.keys()))        

8295


In [0]:
#these concepts will be used as gold labels for concepts which will be related 
#logical format: (e1, e2, rel)
#merge train and test files to create training data:

rel_files = [y for x in os.walk(data_path) for y in glob(os.path.join(x[0], '*.rel'))]
relations = {}
for file in rel_files:
  with open(file, 'r') as f:
    lines = f.readlines()
    for line in lines:
      line = line.replace('\n','')
      if UNCASED_FLAG:
        line = str.lower(line)         
      matches = re.findall(r'"([^"]*)"', line)
      if len(matches) == 3:
        #relations.append((matches[0],matches[2],matches[1]))
        problem = matches[2]
        concept = matches[0]
        rel_class = matches[1]
        if problem not in relations:
          relations[problem] = {}
          relations[problem][concept] = rel_class
        else:
          relations[problem][concept] = rel_class

#print(relations['symptoms'])
#print(relations['symptoms']['overwhelming'])

In [0]:
#create train, test, dev sets, ratio - 3:1:1
#fetch txt files from both folders:
import math
import random
all_text_files = []
for dir in os.listdir(data_path):
  txt_files = [y for x in os.walk(data_path + '/'+dir + '/txt') for y in glob(os.path.join(x[0], '*.txt'))]
  for f in txt_files:
    all_text_files.append(f)

#shuffle data and split:
#random.shuffle(all_text_files)
#num_files = len(all_text_files)

#train_set = all_text_files[0:math.floor(3*num_files/5)]
#dev_set = all_text_files[math.ceil(3*num_files/5):math.floor(4*num_files/5)]
#test_set = all_text_files[math.ceil(4*num_files/5):]

In [0]:
print('Total num of files: {}'.format(len(all_text_files)))

Total num of files: 170


In [0]:
def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll

In [0]:
#use train set to fine-tune BERT models
#read each line and extract the concepts
#preparing training data?

def sentence_getter(relations):

  candidate_sentences = []
  for train in all_text_files:
    #print(train)
    #with open('/content/drive/My Drive/concept_assertion_relation_training_data/beth/txt/record-30.txt', 'r') as f:
    with open(train, 'r') as f:
      lines = f.readlines()
      for line in lines:
        line = line.replace('\n','')
        if UNCASED_FLAG:
          line = str.lower(line)
        line_split = line.split(' ')
        #parse line for conepts and store concept type as well:
        line_concepts_all = [con for con in concept_dict.keys() if ' '+con+' ' in line]
        line_concepts = []
        #test_line = 'on physical exam today , the patient did not have any swelling , and he states that he did not feel the swelling was present .'
        #only keep largest string wherever there is overlapping sequence:
        for con in line_concepts_all:
          is_substring = False
          for s in line_concepts_all:
            if con != s and con in s:
              is_substring = True
          
          if not is_substring:
            line_concepts.append(con)

        #print(line)
        #print(line_concepts)

        #isolate problems in the line:
        if len(line_concepts) >= 2:
          #pull corresponding relations from cache, create new lines with tags from cache
          problems = list(set(line_concepts) & set([con for con in concept_dict.keys() if concept_dict[con] == 'problem']))
          treatments = list(set(line_concepts) & set([con for con in concept_dict.keys() if concept_dict[con] == 'treatment']))
          tests = list(set(line_concepts) & set([con for con in concept_dict.keys() if concept_dict[con] == 'test']))

          '''
          if line == 'there is a persistent flow through an inferior branch of the left anterior descending to pa fistula and a persistent branch of the circumflex to pa branch fistula which anatomically were not suitable for coil embolization .':
            #print(line)
            print(problems)
            print(treatments)
          '''

          for prob1 in problems:

            #making this the main problem for the loop
            if prob1 in relations:
              #fetch permutations of problems:
              for prob2 in problems:
                if prob1 != prob2:

                  rel = relations[prob1].get(prob2,None)
                  if rel == None:
                    continue

                  tag = ['O' for word in line_split]
                  #add tags for problem 1:
                  _split = prob1.split(' ')
                  start, end = find_sub_list(_split, line_split)
                  tag[start] = 'B-problem'
                  for i in range(start + 1, end):
                    tag[i] = 'I-problem'
                    
                  #add tags for problem 2:
                  _split = prob2.split(' ')
                  start, end = find_sub_list(_split, line_split)
                  tag[start] = 'B-problem'
                  for i in range(start + 1, end):
                    tag[i] = 'I-problem'

                  candidate_sentences.append((line, tag, rel))

              #fetch permutations of treatments:
              for treat in treatments:

                  rel = relations[prob1].get(treat, None)
                  #print(rel)
                  '''
                  if line == 'there is a persistent flow through an inferior branch of the left anterior descending to pa fistula and a persistent branch of the circumflex to pa branch fistula which anatomically were not suitable for coil embolization .':
                    print(prob1)
                    print(treat)
                    print(rel)
                  '''

                  if rel == None:
                    continue

                  
                  tag = ['O' for word in line_split]
                  #add tags for problem:
                  _split = prob1.split(' ')
                  start, end = find_sub_list(_split, line_split)
                  tag[start] = 'B-problem'
                  for i in range(start + 1, end):
                    tag[i] = 'I-problem'
                    
                  #add tag for treatment:
                  _split = treat.split(' ')
                  start, end = find_sub_list(_split, line_split)
                  tag[start] = 'B-treatment'
                  for i in range(start + 1, end):
                    tag[i] = 'I-treatment'

                  candidate_sentences.append((line, tag, rel))

              #fetch permutations of tests:
              for test in tests:
                  rel = relations[prob1].get(test, None)
                  #print(rel)
                  if rel is None:
                    continue

                  line_split = line.split()
                  tag = ['O' for word in line_split]
                  #add tags for problem:
                  _split = prob1.split(' ')
                  start, end = find_sub_list(_split, line_split)
                  tag[start] = 'B-problem'
                  for i in range(start + 1, end):
                    tag[i] = 'I-problem'
                    
                  #add tag for test:
                  _split = test.split(' ')
                  start, end = find_sub_list(_split, line_split)
                  tag[start] = 'B-test'
                  for i in range(start + 1, end):
                    tag[i] = 'I-test'
              
                  candidate_sentences.append((line, tag, rel))
          
  return candidate_sentences


In [0]:
transformed_input = sentence_getter(relations)


In [0]:
print(len(transformed_input))

1229


In [0]:
labels = [s[2] for s in transformed_input]
print(set(labels))
print(len(labels))

{'TeCP', 'TrCP', 'TeRP', 'TrAP', 'TrIP', 'PIP', 'TrNAP', 'TrWP'}
1229


In [0]:
# Get the lists of sentences and their labels.
sentences = [s[0].split(' ') for s in transformed_input]
#bio_sentences = [' '.join(w for w in s[1]) for s in transformed_input]
bio_sentences = [s[1] for s in transformed_input]
#modify sentences to include BIO tag to the tokens:
new_sentences = []
for i in range(len(sentences)):
  new_sentences.append([s1 +'_' +s2 for s1,s2 in zip(sentences[i],bio_sentences[i])])

labels = [s[2] for s in transformed_input]
label_vals = list(set(labels))
tag2idx = {t: i for i, t in enumerate(label_vals)}
label_idx = [tag2idx[s] for s in labels]

#release memory
#transformed_input = None

In [0]:
new_sentences[0]

['The_O',
 'patient_O',
 "'s_O",
 'lung_O',
 'examination_B-test',
 'revealed_O',
 'the_O',
 'lungs_O',
 'to_O',
 'be_O',
 'clear_O',
 'to_O',
 'auscultation_O',
 'bilaterally_O',
 ',_O',
 'no_O',
 'crepitus_O',
 ',_O',
 'and_O',
 'a_B-problem',
 'left_I-problem',
 'clavicular_I-problem',
 'deformity_I-problem',
 'was_O',
 'grossly_O',
 'apparent_O',
 'on_O',
 'examination_O',
 '._O']

In [0]:
for label in label_vals:
  _t = [s for s in labels if s == label]
  print('Number of sentences for label : {} is {}'.format(label,len(_t)))

Number of sentences for label : TeCP is 75
Number of sentences for label : TrCP is 83
Number of sentences for label : TeRP is 252
Number of sentences for label : TrAP is 323
Number of sentences for label : TrIP is 24
Number of sentences for label : PIP is 441
Number of sentences for label : TrNAP is 21
Number of sentences for label : TrWP is 10


In [0]:
max_len = 0
sen_lengths = []
# For every sentence...
for sent in new_sentences:

    sent = ' '.join(w for w in sent)
    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))
    sen_lengths.append(len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  352


In [0]:
print(sum(sen_lengths)/len(sen_lengths))

99.9039869812856


In [0]:
#separate train, validation and test set:
# Calculate the number of samples to include in each set.
train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size


# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
print('{:>5,} test samples'.format(test_size))

1,323 training samples
  441 validation samples
  442 test samples


In [0]:
# Tokenize all of the sentences and map the tokens to their word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in new_sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    sent = ' '.join(w for w in sent)
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = MAX_LEN,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(label_idx)

# Print sentence 0, now as a list of IDs.
print('Original: ', new_sentences[0])
print('Token IDs:', input_ids[0])
print('Lables:', labels[0])

Original:  ['The_O', 'patient_O', "'s_O", 'lung_O', 'examination_B-test', 'revealed_O', 'the_O', 'lungs_O', 'to_O', 'be_O', 'clear_O', 'to_O', 'auscultation_O', 'bilaterally_O', ',_O', 'no_O', 'crepitus_O', ',_O', 'and_O', 'a_B-problem', 'left_I-problem', 'clavicular_I-problem', 'deformity_I-problem', 'was_O', 'grossly_O', 'apparent_O', 'on_O', 'examination_O', '._O']
Token IDs: tensor([  101,  1109,   168,   152,  5351,   168,   152,   112,   188,   168,
          152, 13093,   168,   152,  8179,   168,   139,   118,  2774,  3090,
          168,   152,  1103,   168,   152,  8682,   168,   152,  1106,   168,
          152,  1129,   168,   152,  2330,   168,   152,  1106,   168,   152,
        12686,  1116, 10182, 23934,  2116,   168,   152, 20557,  1193,   168,
          152,   117,   168,   152,  1185,   168,   152,   172,  1874, 18965,
         1361,   168,   152,   117,   168,   152,  1105,   168,   152,   170,
          168,   139,   118,  2463,  1286,   168,   146,   118,  2463,  

In [0]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size


# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
print('{:>5,} test samples'.format(test_size))

  737 training samples
  245 validation samples
  247 test samples


In [0]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

#will implement test case below

In [0]:
#optimizer:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [0]:
import numpy as np
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [0]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        #print(step)
        if step % 20 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...


RuntimeError: ignored

In [0]:
#run on test set now:
test_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [0]:
!pip install seqeval

In [0]:
# Prediction on test set
from seqeval.metrics import f1_score, precision_score, recall_score
print('Predicting labels for {:,} test sentences...'.format(len(test_dataset)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in test_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)
  #predictions.extend([list(p) for p in np.argmax(logits, axis=1)])
  #true_labels.append(label_ids)
  

#pred_tags = [[label_vals[p_i] for p_i in p] for p in predictions]
#valid_tags = [[label_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
#print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))
print('DONE.')

In [0]:
# For each input batch...
preds = []
labels = []
for i in range(len(true_labels)):
  
  # The predictions for this batch are a 2-column ndarray (one column for "0" 
  # and one column for "1"). Pick the label with the highest value and turn this
  # in to a list of indices from the labels list
  pred_labels_i = np.argmax(predictions[i], axis = 1)
  preds.extend(pred_labels_i)
  labels.extend(true_labels[i])
 
 
#calculate F1 score:
pred_tags = [label_vals[p] for p in preds]
valid_tags = [label_vals[l_i] for l in true_labels for l_i in l]
#print(pred_tags[0:10])
#print(valid_tags[0:10])
print('Results for model :{} '.format(BERT_MODEL))
print('Precision:{}'.format(precision_score(valid_tags,pred_tags)))
print('Recall:{}'.format(recall_score(valid_tags,pred_tags)))
print('F1 score: {}'.format(f1_score(valid_tags, pred_tags, average = "macro")))

In [0]:
!pip install sklearn

In [0]:
from sklearn.metrics import f1_score, precision_score, recall_score

# Prediction on test set vals
print('Predicting labels for {:,} test sentences...'.format(len(test_dataset)))

# Put model in evaluation mode
model.eval()

# Predict for each category:
for label in label_vals:
  
  #run on test sub set now:
  test_data_subset = [t for t in dataset if t[2] == torch.tensor(tag2idx[label])]
  #print('Length of first tensor in subset: {}'.format(len(test_data_subset[0])))
  test_subset_dataloader = DataLoader(
            test_data_subset, # The test samples for the given label
            sampler = SequentialSampler(test_data_subset), # Pull out batches sequentially.
            batch_size = 5 # Evaluate with this batch size. #keeping it small as label exapmples are less
        )

  print('Number of examples for label:{} is {}'.format(label,len(test_data_subset)))

  # Tracking variables 
  predictions , true_labels = [], []
  for batch in test_subset_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

  # For each input batch...
  preds = []
  labels = []
  for i in range(len(true_labels)):
  
    # The predictions for this batch are a 2-column ndarray (one column for "0" 
    # and one column for "1"). Pick the label with the highest value and turn this
    # in to a list of indices from the labels list
    pred_labels_i = np.argmax(predictions[i], axis = 1)
    preds.extend(pred_labels_i)
    labels.extend(true_labels[i])
 
 
  #calculate F1 score:
  _pred_tags = [label_vals[p] for p in preds]
  valid_tags = [1 for l in true_labels for l_i in l]
  pred_tags = []
  for p in _pred_tags:
    if p == label:
      pred_tags.append(1)
    else:
      pred_tags.append(0)

  #print('Length of pred tags = {} and valid tags = {}'.format(len(pred_tags), len(valid_tags)))
  print('Results for model :{} , category :{} '.format(BERT_MODEL,label))
  print('Precision:{}'.format(precision_score(valid_tags,pred_tags)))
  print('Recall:{}'.format(recall_score(valid_tags,pred_tags)))
  print('F1 score: {}'.format(f1_score(valid_tags, pred_tags)))

In [0]:
# Prediction on label type
from seqeval.metrics import f1_score, precision_score, recall_score
print('Predicting labels for {:,} test sentences...'.format(len(test_dataset)))

# Put model in evaluation mode
model.eval()

type_labels = ['test-problem','treatment-problem','problem-problem']
type_label_dict = {}
for _type in type_labels:
  type_label_dict[_type] = []

lable_type_dict = {}
#for each label build dictionary
for label in label_vals:  
  if 'te' in str.lower(label):
    type_label_dict['test-problem'].append(torch.tensor(tag2idx[label]))
    lable_type_dict[label] = 'test-problem'
  elif 'tr' in str.lower(label):
    type_label_dict['treatment-problem'].append(torch.tensor(tag2idx[label]))
    lable_type_dict[label] = 'treatment-problem'
  else:
    type_label_dict['problem-problem'].append(torch.tensor(tag2idx[label]))
    lable_type_dict[label] = 'problem-problem'


# Predict for each category:
for label in type_labels:
  #corresponding label:
  #run on test sub set now:
  test_data_subset = [t for t in dataset if t[2] in type_label_dict[label]]
  #print('Length of first tensor in subset: {}'.format(len(test_data_subset[0])))
  test_type_dataloader = DataLoader(
            test_data_subset, # The test samples for the given label
            sampler = SequentialSampler(test_data_subset), # Pull out batches sequentially.
            batch_size = 5 # Evaluate with this batch size. #keeping it small as label exapmples are less
        )

  print('Number of examples for label:{} is {}'.format(label,len(test_data_subset)))

  # Tracking variables 
  predictions , true_labels = [], []
  for batch in test_type_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

  # For each input batch...
  preds = []
  labels = []
  for i in range(len(true_labels)):
  
    # The predictions for this batch are a 2-column ndarray (one column for "0" 
    # and one column for "1"). Pick the label with the highest value and turn this
    # in to a list of indices from the labels list
    pred_labels_i = np.argmax(predictions[i], axis = 1)
    preds.extend(pred_labels_i)
    labels.extend(true_labels[i])
 
 
  #calculate F1 score:
  #pred_tags = [label_vals[p] for p in preds]
  #valid_tags = [label_vals[l_i] for l in true_labels for l_i in l]
  #print(pred_tags[0:10])
  #print(valid_tags[0:10])
  pred_tags = [lable_type_dict[label_vals[p]] for p in preds]
  valid_tags = [lable_type_dict[label_vals[l_i]] for l in true_labels for l_i in l]
  #print(pred_tags[0:10])
  #print(valid_tags[0:10])
  print('Results for model :{} , Type :{} '.format(BERT_MODEL,label))
  print('Precision:{}'.format(precision_score(valid_tags,pred_tags, average = "macro")))
  print('Recall:{}'.format(recall_score(valid_tags,pred_tags, average = "macro")))
  print('F1 score: {}'.format(f1_score(valid_tags, pred_tags, average = "macro")))